# Finite strains

A quick notebook to try to reduce the problem to looking at some simple pictures. There seems to be a lot more than one way to define the strain tensor components. We use a 2D example for now so it is easier to draw pictures. This is a part two which attempts to get to the point of choosing between some options of different strain tensors.

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as pl, numpy as np

## Reference State

Use the case of an in-situ loading experiment where the initial state is known. 
This is also like the case of mapping around a crack tip in a deforming single crystal.
It will not work for residual strain where there is no reference orientation.
We will assume the crystal is cubic (square) and was carefully aligned on the diffractometer
so the axes are parallel to the axes. The unit cell and lattice vectors in real space are:

In [ ]:
def unstrained_lattice(a0, f):
    a = ( a0,  0,  0 )
    b = (  0, a0,  0 )
    c = ( 0,  0, a0 )
    return a, b, c

## Pure shear strain
We apply a pure shear strain as drawn in some pictures found on the internet. 
The atoms which were on the laboratory axes move at exactly 90 degrees to the axes and not
along the axes. We have "f" as symbol for the fractional finite movement. 
It is not clear whether this "f" corresponds to a shear strain $\epsilon_{12}$

In [ ]:
def pure_shear_lattice(a0, f):
    a = (  a0, f*a0,  0)  # vector in x/y/z laboratory space
    b = (f*a0,   a0,  0)
    c = (   0,    0, a0)
    return (a, b, c)

## Simple shear strain
This case does not seem simple at all. Nevertheless, 
there are pictures on the internet showing this definition.
The loss of symmetry is appalling. We will re-use the same f as before but we are choosing to keep the x-axis parallel to the a axis. A different choice would place it along b.

In [ ]:
def simple_shear_lattice(a0, f):
    a = (     a0,      0,  0)
    b = ( 2*f*a0,     a0,  0)
    c = (      0,      0, a0)
    return (a, b, c)

## Experimental shear strain
We imagine a careful experimentalist who realigns their strained crystal on the diffractometer after is has undergone a **pure** strain. Their crystal is beautiful and symmetric but because the realign it this is no longer to easy to see. The lattice vectors are now harder to compute:

In [ ]:
def experimental_shear_lattice(a0, f):
    a1 = a0 * np.sqrt(1+f*f)   # length of a and b after straining
    phi = 2 * np.arctan(f)  # Rotation angle to realign crystal
    a = ( a1, 0, 0 )
    b = ( a1*np.sin(phi), a1*np.cos(phi), 0 )
    c = ( 0, 0, a0 )
    return (a, b, c)

## A new Strain Tensor?? What to do here ??
So the "left Cauchy-Green" or "Finger" deformation tensor is the one where you first rotate the crystal and then deform it at the end of the process. It doesn't matter if you put the sample on upside down by accident. This seems like overwhelmingly a better choice for the case of samples that have been dropped. Wikipedia has it as **B**. They have the inverse as **c** and call that the Piola tensor or Finger tensor. In the finite strain tensors section they have a thing they call the Euler-Almansi finite strain tensor reference to the deformed configuration, i.e. the Eulerian description:

$ \mathbf{e} = \frac{1}{2} (\mathbf{I} -\mathbf{c}) = \frac{1}{2}( \mathbf{I} - \mathbf{B}^{-1}) $

This is the intention of the function `EulerianAlmansiFiniteStrain`

Numerically these are less pretty because of the difference between:

$ \epsilon = \frac{d - d0}{d0} \neq \frac{d - d0}{d} $

We can compute a thing which looks like this by analogy to the finite strain literature that suggests anything goes. Because We are putting maths together in the way toddlers to jigsaw puzzles this will be named `NotAGoodIdea` in the code.

$ \mathbf{e} = \frac{1}{2}( \mathbf{B} - \mathbf{I} ) $

...but this is not found on wikipedia. As such, I guess the mechanics people will prefer something else. If we take the right-Cauchy-Green option it depends on the initial or reference orientation in the crystal. We will see if we can manage to do it anyway:

$ \mathbf{e} = \frac{1}{2}( \mathbf{C} - \mathbf{I} ) $

Following wikipedia (?!?) we have $\mathbf{C} = \mathbf{F^T F} $ and $\mathbf{B} = \mathbf{B^{-1}}$.

In what follows: note that **ubi**$ = (UB)^{-1} = B^{-1}U^{-1} $ so that whenever we use this matrix we want to try ensure the transpose is done to keep the $U$ out of the results. Many hours have been spent with transposes the wrong way around, etc. A seemingly reasonable guess is based on a idea that it takes the reference state to the final state, so that:

$ \mathbf{M_1} = \mathbf{ F } \mathbf{M_0} $

$ \mathbf{F} = \mathbf{ M_1} \mathbf{M_0^{-1}} $

We are picking the **ubi** matrix from the software which is the $\mathbf{a,b,c}$ lattice vectors as the thing to use as $\mathbf{M}$. Clearly, this is a guess that might not be right.

In [ ]:
from ImageD11.unitcell import unitcell
import xfab.tools

def GreenLagrangian_A(ubi, reference_cell):
    reciprocal_basis = np.linalg.inv( xfab.tools.form_a_mat(reference_cell) )
    # ubi = B-1 U-1 so transpose:
    F = np.dot( ubi.T, reciprocal_basis ) # U.B-1.Bo
    C = np.dot(F.T, F)
    E = 0.5*(C-np.eye(3))
    return E

def GreenLagrangian_B(ubi, reference_cell):
    reciprocal_basis = xfab.tools.form_b_mat(reference_cell)/2/np.pi
    F = np.dot( ubi.T, reciprocal_basis )
    C = np.dot(F.T, F)
    E = 0.5*(C-np.eye(3))
    return E

def EulerianAlmansiFiniteStrain(ubi, reference_cell):
    """
    Compute a Finite Strain Tensor for a grain
    with ImageD11 indexing matrix UBI == [a,b,c] lattice vectors
    in real space. The strain is with reference to a 
    reference_cell = [a,b,c,alpha,beta,gamma]
    We suppress the effect of rotation and try to copy an 
    equation from wikipedia
    """
    reciprocal_metric_tensor = unitcell(reference_cell).gi
    # Deformation Gradient Tensor:
    # F = np.dot( ubi, reciprocal_basis )
    # Left Cauchy Green
    # B = np.dot( F, F.T )
    B = np.dot( np.dot( ubi.T, reciprocal_metric_tensor ), ubi)
    e = 0.5*(np.eye(3)-np.linalg.inv(B))
    return e

def NotAGoodIdea(ubi, reference_cell):
    """
    Compute a Finite Strain Tensor for a grain
    with ImageD11 indexing matrix UBI == [a,b,c] lattice vectors
    in real space. The strain is with reference to a 
    reference_cell = [a,b,c,alpha,beta,gamma]
    We suppress the effect of rotation but reference to an
    initial state (e.g. (d-d0)/d0 rather than (d-d0)/d)
    """
    reciprocal_metric_tensor = unitcell(reference_cell).gi
    # Deformation Gradient Tensor:
    # F = np.dot( ubi, reciprocal_basis )
    # Left Cauchy Green
    # B = np.dot( F, F.T )
    B = np.dot( np.dot( ubi.T, reciprocal_metric_tensor ), ubi)
    e = 0.5*(B - np.eye(3))
    return e


## What do these look like?
We will plot the three lattices on three plots for some different values of f.

In [ ]:
from ImageD11.grain import grain

def plot_lattice( ubi, ax ):
    # walk around the x/y square:
    points = [ (0,0,0), (1,0,0), (1,1,0), (0,1,0), (0,0,0) ]
    x,y,z = np.dot( points, ubi).T
    ax.plot( x, y, "o-" )
    ax.text(x[1],y[1],'a')
    ax.text(x[-2],y[-2],'b')
    ax.set_xlabel("x")
    ax.set_aspect("equal")
    
def plotstrain(lattice, a0, f, name, reference_cell):
    g0 = grain( np.linalg.inv(unitcell(reference_cell).B) )
    ubi0 = g0.ubi
    g = grain( lattice(a0,f) )
    fig, ax = pl.subplots(1,1,
                          figsize=(4,4))
    plot_lattice( ubi0 , ax )
    plot_lattice( g.ubi, ax )
    ax.set_title("%.2f %s"%(f, name))
    pl.show()
    np.set_printoptions(floatmode='fixed', precision=6, suppress=True)            
    print(name,"unit cell",g.unitcell)
    print("dzero unit cell",g0.unitcell)
    print("UBI:")
    print(g.ubi)
    print("UBI0:")
    print(g0.ubi)
    np.set_printoptions(floatmode='fixed', precision=1, suppress=True)
    print("Grain co-ordinates micro eps")
    print(g.eps_grain_matrix(g0.unitcell)*1e6)
    print("Sample co-ordinates micro eps")
    print(g.eps_sample_matrix(g0.unitcell)*1e6)    
    EAFS  = EulerianAlmansiFiniteStrain( g.ubi, g0.unitcell )
    print("Euler Almansi Finite Strain")
    print(EAFS*1e6)
    NAGI  = NotAGoodIdea( g.ubi, g0.unitcell )
    print("Not A Good Idea based strain")
    print(NAGI*1e6)
    print("GreenLagrangian_A")
    print(GreenLagrangian_A(g.ubi, g0.unitcell)*1e6)
    print("GreenLagrangian_B")
    print(GreenLagrangian_B(g.ubi, g0.unitcell)*1e6)
    
    
a0 = 4.
f = 0.1
reference_cell = [a0,a0,a0,90,90,90]

def flipped(a0,f):
    a, b, c = np.array( simple_shear_lattice( a0, f ) )
    return (b, a, -c )
    
lattices = {
    "Pure Shear" : pure_shear_lattice,
    "Experimental pure" : experimental_shear_lattice,
    "Simple Shear" : simple_shear_lattice,
    "Flipped Simple" : flipped
}    

for name in lattices:
    print("============================================================")
    plotstrain(lattices[name], a0, f, name, reference_cell)

Now we bring the reference cell closer to distorted

In [ ]:
a0 = 4.
f = 0.1
reference_cell = [a0,a0,a0,90,90,79.]

def flipped(a0,f):
    a, b, c = np.array( simple_shear_lattice( a0, f ) )
    return (b, a, -c )
    
lattices = {
    "pure" : pure_shear_lattice,
    "experimental pure" : experimental_shear_lattice,
    "simple" : simple_shear_lattice,
    "flipped" : flipped
}    

for name in lattices:
    print("============================================================")
    plotstrain(lattices[name], a0, f, name, reference_cell)